In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import date, timedelta, datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_today():
    now = date.today()
    now = now.strftime("%Y-%m-%d")
    return now

In [ ]:
df = pd.read_csv("ctp_after.csv")
# df = pd.read_csv("after.csv", dtype={"Fips":str}, parse_dates=['ReportDate'])
df.head()

In [ ]:
# df["PrevPos7DayRegion"] = df["Positivity7DayRegion"]
# df["PrevPos7DayRegion"] = df.apply(lambda row:  df[df.ReportDate == prev]["Positivity7DayRegion"], axis=1)
# df.loc[df[df.ReportDate == prev],"PrevPos7DayRegion"]

df["Pos7DayRegion_13"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-13))
df["Pos7DayRegion_12"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-12))
df["Pos7DayRegion_11"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-11))
df["Pos7DayRegion_10"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-10))
df["Pos7DayRegion_09"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-9))
df["Pos7DayRegion_08"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-8))
df["Pos7DayRegion_07"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-7))
df["Pos7DayRegion_06"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-6))
df["Pos7DayRegion_05"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-5))
df["Pos7DayRegion_04"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-4))
df["Pos7DayRegion_03"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-3))
df["Pos7DayRegion_02"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-2))
df["Pos7DayRegion_01"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(-1))
df["Pos7DayRegion_00"] = df.groupby(["CensusRegionName", "State"])["Positivity7DayRegion"].transform(lambda x: x.shift(0))

df.head()


In [ ]:
df["TrendRegion"] = np.nan
df.loc[(df["Positivity7DayRegion"] - df["Pos7DayRegion_13"]) >  0.1, "TrendRegion"] = "Increasing"
df.loc[(df["Positivity7DayRegion"] - df["Pos7DayRegion_13"]) < -0.1, "TrendRegion"] = "Decreasing"
df["TrendRegion"].replace(np.nan, 'Flat', regex=True, inplace=True)
df.head()
# df.tail()

In [ ]:
df = df[df.ReportDate > "2020-03-31 00:00:00"]

In [ ]:
groups = df.groupby(["CensusRegionName","State"])
keep_cols = ["ReportDate","CensusRegionName","TotTestIncrRegion","Positivity7DayRegion",  
             'Pos7DayRegion_13', 'Pos7DayRegion_12', 'Pos7DayRegion_11',
             'Pos7DayRegion_10', 'Pos7DayRegion_09', 'Pos7DayRegion_08',
             'Pos7DayRegion_07', 'Pos7DayRegion_06', 'Pos7DayRegion_05',
             'Pos7DayRegion_04', 'Pos7DayRegion_03', 'Pos7DayRegion_02',
             'Pos7DayRegion_01', 'Pos7DayRegion_00', 'TrendRegion']
west = groups[keep_cols].get_group(("West","Alaska"))
midwest = groups[keep_cols].get_group(("Midwest","Illinois"))
south = groups[keep_cols].get_group(("South","Alabama"))
northeast = groups[keep_cols].get_group(("Northeast","Connecticut"))

In [ ]:
# west.head()

In [ ]:
# midwest.head()

In [ ]:
# south.head()

In [ ]:
# northeast.head()

In [ ]:
fig = make_subplots(rows=1, cols=4,
                    subplot_titles=("Northeast","Midwest","South","West",),
                    specs=[[{"secondary_y": True},{"secondary_y": True},
                           {"secondary_y": True},{"secondary_y": True}]],
                    horizontal_spacing = 0.025,
                    shared_xaxes=True,)


In [ ]:
fig.add_trace(
    go.Bar(x=northeast["ReportDate"], y=northeast["TotTestIncrRegion"], 
          hovertemplate='%{y:10,.1d}', 
          name="Number of Tests",
          opacity=1.0, marker_color='rgb(26, 118, 255)'),
    row=1, col=1, secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=northeast["ReportDate"], y=northeast["Positivity7DayRegion"],
               hovertemplate='%{y:17,.1f}%',
               name="Positivity",
              line=go.scatter.Line(color="black")),
    row=1, col=1, secondary_y=True,
)


fig.update_yaxes(title_text="Daily Tests", range=[0, 375000], tick0=0.0, dtick=25000, visible=True, 
                 showticklabels=True, row=1, col=1, secondary_y=False)
_ = fig.update_yaxes(range=[0, 40], tick0=0.0, dtick=5, visible=False, showticklabels=False, 
                 row=1, col=1, secondary_y=True)


In [ ]:
fig.add_trace(
    go.Bar(x=midwest["ReportDate"], y=midwest["TotTestIncrRegion"], 
          hovertemplate='%{y:10,.1d}', 
          name="Number of Tests",
          opacity=1.0,marker_color='rgb(26, 118, 255)',),
    row=1, col=2, secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=midwest["ReportDate"], y=midwest["Positivity7DayRegion"],
               hovertemplate='%{y:17,.1f}%',
               name="Positivity",
              line=go.scatter.Line(color="black")),
    row=1, col=2, secondary_y=True,
)


fig.update_yaxes(range=[0, 375000], tick0=0.0, dtick=25000, visible=True, showticklabels=False, 
                 row=1, col=2, secondary_y=False)

_ = fig.update_yaxes(range=[0, 40], tick0=0.0, dtick=5, visible=False, showticklabels=False, 
                 row=1, col=2, secondary_y=True)


In [ ]:
fig.add_trace(
    go.Bar(x=south["ReportDate"], y=south["TotTestIncrRegion"], 
          hovertemplate='%{y:10,.1d}', 
          name="Number of Tests",
          opacity=1.0,marker_color='rgb(26, 118, 255)',),
    row=1, col=3, secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=south["ReportDate"], y=south["Positivity7DayRegion"],
               hovertemplate='%{y:17,.1f}%',
               name="Positivity",
              line=go.scatter.Line(color="black")),
    row=1, col=3, secondary_y=True,
)

fig.update_yaxes(range=[0, 375000], tick0=0.0, dtick=25000, visible=True, showticklabels=False, 
                 row=1, col=3, secondary_y=False)

_ = fig.update_yaxes(range=[0, 40], tick0=0.0, dtick=5, visible=False, showticklabels=False, 
                 row=1, col=3, secondary_y=True)

In [ ]:
fig.add_trace(
    go.Bar(x=west["ReportDate"], y=west["TotTestIncrRegion"], 
          hovertemplate='%{y:10,.1d}', 
          name="Number of Tests",
          opacity=1.0,marker_color='rgb(26, 118, 255)',),
    row=1, col=4, secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=west["ReportDate"], y=west["Positivity7DayRegion"],
               hovertemplate='%{y:17,.1f}%',
               name="Positivity",
              line=go.scatter.Line(color="black")),
    row=1, col=4, secondary_y=True,
)



fig.update_yaxes(range=[0, 375000], tick0=0.0, dtick=25000, visible=True, showticklabels=False, 
                 row=1, col=4, secondary_y=False)

_ = fig.update_yaxes(title_text="Percent Positive", range=[0, 40], tick0=0.0, dtick=5, visible=True, 
                 showticklabels=True, showgrid=False, row=1, col=4, secondary_y=True)

In [ ]:
fig.update_layout(
    title_text="Regional Daily Tests and Positivity, 7-Day Rolling Avg.",
    title_x = 0.5,
    title_y = .98,
#     xaxis=dict(range = [startdate, enddate], tickformat = '%b %d', dtick = 14*86400000.0)
#     xaxis=dict(tickformat = '%b %e')
)

# fig.update_xaxes(title_text="Date")

fig.update_xaxes(tickangle=90,tickformat = '%b %e')


fig.update_layout(showlegend=False, bargap=0) 

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=12, 
        font_family="Times"
    )
)

fig.update_layout(hovermode='x unified')

fig.update_layout(
    autosize=False,
    width=1100,
    height=600,
    margin=dict(
        l=90,
        r=0,
        b=50,
        t=100,
        pad=4
    ),
)

# fig.update_xaxes(rangeslider_visible=True)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=14, label="2w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_annotations(y=1.1, selector={'text':'Northeast'})
fig.update_annotations(y=1.1, selector={'text':'Midwest'})
fig.update_annotations(y=1.1, selector={'text':'South'})
fig.update_annotations(y=1.1, selector={'text':'West'})


fig.show()

https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4

In [ ]:
# def zoom(layout, xrange):
#     in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
#     fig.layout.yaxis.range = [in_view.Positivity7DayRegion.min() - 5, in_view.Positivity7DayRegion.max() + 5]

# fig.layout.on_change(zoom, 'xaxis.range')

# fig.show()

In [ ]:
today = get_today()
fig.write_html(f"./regional_positivity_over_time_{today}.html")

In [ ]:
curr = max(df.ReportDate)
curr = datetime.strptime(curr, '%Y-%m-%d')

prev = curr - timedelta(days=13)

numdays = 14
date_list = [prev + timedelta(days=x) for x in range(numdays)]
# date_list

In [ ]:
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Northeast","Midwest","South","West",),
                    specs=[[{"secondary_y": False},{"secondary_y": False}],
                           [{"secondary_y": False},{"secondary_y": False}]],
                    horizontal_spacing = 0.025,
                    )

In [ ]:
y = northeast.iloc[:1,4:-1].values.tolist()[0]
start = f"{y[0]} %" 
end = f"{y[-1]} %"
maxy = round(max(y))
miny = round(min(y))
diff = (maxy - miny - 5)/2.0
maxy = maxy + abs(diff)
miny = miny - abs(diff)

fig.add_trace(
    go.Scatter(
        x=date_list,
        y=y,
        hovertemplate = '%{y:.2f}%',
        name="",
        line=go.scatter.Line(color="blue"),
        mode="lines+markers+text",
        text=[start, "", "", "", "", "", "", "", "", "", "", "", "", end],
        textposition="bottom center"
    ),
    row=1, col=1
)

_ = fig.update_yaxes(title_text="Positivity", visible=True, range=[miny, maxy],
                 tick0=miny, dtick=(maxy-miny)/5,
                 showticklabels=True, row=1, col=1, secondary_y=False)



In [ ]:
y = midwest.iloc[:1,4:-1].values.tolist()[0]
start = f"{y[0]} %" 
end = f"{y[-1]} %"
maxy = round(max(y))
miny = round(min(y))
diff = (maxy - miny - 5)/2.0
maxy = maxy + abs(diff)
miny = miny - abs(diff)

fig.add_trace(
    go.Scatter(
        x=date_list,
        y=y,
        hovertemplate = '%{y:.2f}%',
        name="",
        line=go.scatter.Line(color="blue"),
        mode="lines+markers+text",
        text=[start, "", "", "", "", "", "", "", "", "", "", "", "", end],
        textposition="bottom center"
    ),
    row=1, col=2
)

_ = fig.update_yaxes( visible=True, range=[miny, maxy],
                 tick0=miny, dtick=(maxy-miny)/5,
                 showticklabels=True, row=1, col=2, secondary_y=False)



In [ ]:
y = south.iloc[:1,4:-1].values.tolist()[0]
start = f"{y[0]} %" 
end = f"{y[-1]} %"
maxy = round(max(y))
miny = round(min(y))
diff = (maxy - miny - 5)/2.0
maxy = maxy + abs(diff)
miny = miny - abs(diff)

fig.add_trace(
    go.Scatter(
        x=date_list,
        y=y,
        hovertemplate = '%{y:.2f}%',
        name="",
        line=go.scatter.Line(color="blue"),
        mode="lines+markers+text",
        text=[start, "", "", "", "", "", "", "", "", "", "", "", "", end],
        textposition="bottom center"
    ),
    row=2, col=1
)

_ = fig.update_yaxes(title_text="Positivity", visible=True, range=[miny, maxy],
                 tick0=miny, dtick=(maxy-miny)/5,
                 showticklabels=True, row=2, col=1, secondary_y=False)

_ = fig.update_xaxes(tickformat = '%b %e')

In [ ]:
y = west.iloc[:1,4:-1].values.tolist()[0]
start = f"{y[0]} %" 
end = f"{y[-1]} %"
maxy = round(max(y))
miny = round(min(y))
diff = (maxy - miny - 5)/2.0
maxy = maxy + abs(diff)
miny = miny - abs(diff)

fig.add_trace(
    go.Scatter(
        x=date_list,
        y=y,
        hovertemplate = '%{y:.2f}%',
        name="",
        line=go.scatter.Line(color="blue"),
        mode="lines+markers+text",
        text=[start, "", "", "", "", "", "", "", "", "", "", "", "", end],
        textposition="bottom center"
    ),
    row=2, col=2
)

_ = fig.update_yaxes( visible=True, range=[miny, maxy],
                 tick0=miny, dtick=(maxy-miny)/5,
                 showticklabels=True, row=2, col=2, secondary_y=False)

In [ ]:
fig.update_layout(hovermode='x',  xaxis=dict(tickformat = '%b %e'), showlegend=False, 
                  title="Positivity by Region: 2-Week Trend",title_x = 0.5,)
fig.show()

In [ ]:
today = get_today()
fig.write_html(f"./regional_positivity_2week_trend_{today}.html")